In [1]:
from MuTe_RL.simulations import rubiks
from MuTe_RL.simulations.rubiks import cube

from MuTe_RL.models import rubiks as model_rubiks
from MuTe_RL.models.rubiks import look_ahead

from importlib import reload
reload(cube), reload(rubiks), reload(model_rubiks), reload(look_ahead)

from MuTe_RL.simulations.rubiks import Cube
from MuTe_RL.models.rubiks import search_best_paths, score_actions

from IPython.display import clear_output
from copy import deepcopy
import numpy as np
from time import sleep
None

/Users/mudassar.iqbal/.virtualenvs/mute/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Simulation

In [2]:
c = Cube(n_layers=5, numbered=True)
# check for impossible corners or edge pieces after shuffle
print(c)
c.move("x1")
c.move("y1")
c.move("x1'")
c.move("y1'")

               0  1  2  3  4 
               5  6  7  8  9 
               10 11 12 13 14
               15 16 17 18 19
               20 21 22 23 24
0  1  2  3  4  0  1  2  3  4  0  1  2  3  4  0  1  2  3  4 
5  6  7  8  9  5  6  7  8  9  5  6  7  8  9  5  6  7  8  9 
10 11 12 13 14 10 11 12 13 14 10 11 12 13 14 10 11 12 13 14
15 16 17 18 19 15 16 17 18 19 15 16 17 18 19 15 16 17 18 19
20 21 22 23 24 20 21 22 23 24 20 21 22 23 24 20 21 22 23 24
               0  1  2  3  4 
               5  6  7  8  9 
               10 11 12 13 14
               15 16 17 18 19
               20 21 22 23 24



               0  1  2  3  4 
               5  6  7  8  9 
               10 11 12 13 14
               15 16 17 18 19
               20 21 22 23 24
0  1  2  3  4  0  1  2  3  4  0  1  2  3  4  0  1  2  3  4 
5  6  7  8  9  5  6  7  8  9  5  6  7  8  9  5  6  7  8  9 
10 11 12 13 14 10 11 12 13 14 10 11 12 13 14 10 11 12 13 14
15 16 17 18 19 15 16 17 18 19 15 16 17 18 19 15 16 17 6  19
20 21 22 23 24 20 21 22 23 24 20 21 22 23 24 20 21 22 23 24
               0  1  2  3  4 
               5  18 7  8  9 
               10 11 12 13 14
               15 16 17 18 19
               20 21 22 23 24

In [9]:
c.shuffle().fraction_solved()

0.16666666666666666

In [15]:
[
    f"{axis}{j}"
    for i in range(0,int((n+1)/2))

    # add symitry
    for j in {i, n-1-i}
    for axis in ["x","y",""]

    if j not in [int(n/2)]
]

['x0', 'y0', 'x6', 'y6', 'x1', 'y1', 'x5', 'y5', 'x2', 'y2', 'x4', 'y4']

In [18]:
n = 7
kyoob = Cube(n_layers=n, numbered=False, disabled_layers={
    f"{axis}{j}"
    for i in range(1,int((n+1)/2))

    # add symmetry
    for j in {i, n-1-i}
    for axis in ["x", "y", "z"]

    if j not in [int(n/2)]
})
kyoob.shuffle(), kyoob.fraction_solved()

(              ■ ■ ■ ■ ■ ■ ■
               ■ ■ ■ ■ ■ ■ ■
               ■ ■ ■ ■ ■ ■ ■
               ■ ■ ■ ■ ■ ■ ■
               ■ ■ ■ ■ ■ ■ ■
               ■ ■ ■ ■ ■ ■ ■
               ■ ■ ■ ■ ■ ■ ■
 ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■
 ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■
 ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■
 ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■
 ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■
 ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■
 ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■
               ■ ■ ■ ■ ■ ■ ■
               ■ ■ ■ ■ ■ ■ ■
               ■ ■ ■ ■ ■ ■ ■
               ■ ■ ■ ■ ■ ■ ■
               ■ ■ ■ ■ ■ ■ ■
               ■ ■ ■ ■ ■ ■ ■
               ■ ■ ■ ■ ■ ■ ■,
 0.14814814814814814)

In [9]:
print(kyoob.possible_moves())

['x0', "x0'", 'y0', "y0'", 'z0', "z0'", 'x3', "x3'", 'y3', "y3'", 'z3', "z3'", 'x6', "x6'", 'y6', "y6'", 'z6', "z6'"]


In [41]:
c.to_vector()

[1, 3, 3, 2, 1, 5, 2, 5, 2, 4, 5, 2, 6, 4, 1, 4, 3, 4, 3, 1, 6, 6, 5, 6]

In [3]:
c.reset().move("x0").move("x0'").is_solved()

True

In [16]:
c.reset().shuffle().fraction_solved()

0.2916666666666667

In [7]:
# z3 y1 y0 z3' x2' y3 y3' --> consecutive opposites popped
c.moves_stack

['z3', 'y1', 'y0', "z3'", "x2'"]

In [3]:
# entire cube rotation
Cube(n_layers=3, numbered=True).move("z0").move("z1").move("z2").fraction_solved()

0.0

# Models

In [16]:
n=5
c = Cube(n_layers=n, numbered=True, disabled_layers = {
    f"{axis}{j}"
    for i in range(1,int((n+1)/2))

    # add symmetry
    for j in {i, n-1-i}
    for axis in ["x", "y", "z"]

#     if j not in [int(n/2)]
})
moves = c.possible_moves*10
look_ahead.random.shuffle(moves)
for move_id in moves[:15]:
        c.move(move_id)
c

               24 15 10 5  4 
               21 16 11 6  9 
               22 17 12 7  14
               23 18 13 8  19
               24 19 14 9  24
20 9  14 19 4  20 15 10 5  4  0  15 10 5  0  0  1  2  3  20
19 6  7  8  23 3  8  13 18 23 21 6  7  8  1  3  18 17 16 15
14 11 12 13 22 2  7  12 17 22 22 11 12 13 2  2  13 12 11 10
9  16 17 18 21 1  6  11 16 21 23 16 17 18 3  1  8  7  6  5 
24 19 14 9  24 24 21 22 23 20 0  5  10 15 4  0  3  2  1  0 
               20 15 10 5  4 
               3  16 11 6  9 
               2  17 12 7  14
               1  18 13 8  19
               20 21 22 23 4 

In [3]:
def remove_cycles(actions):
    def complementary(action_1, action_2):
        return action_1.strip("'") == action_2.strip("'") and (
                (action_1[-1] == "'" and action_2[-1] != "'")
                or (action_1[-1] != "'" and action_2[-1] == "'")
            )

    if (
        len(actions) > 1 and
        # opposite moves
        complementary(actions[-1], actions[-2])
    ):
        actions = actions[:-2]
    if (
        len(actions) > 3 and
        # 360 degree rotation
        len(set(actions[-4:]))==1
    ):
        actions = actions[:-4]

    return actions

In [18]:
print(c.moves_stack)
solutions = search_best_paths(
    state_0 = c,
    actions = c.possible_moves,
    get_next_state = lambda cube, action: deepcopy(cube).move(action),
    get_state_score = lambda cube: cube.fraction_solved(),
    remove_action_cycles = remove_cycles,
    depth_decay=0.995,
    n_branches=round(len(c.possible_moves) ** 2),
    max_depth=13,
    greedy_fraction = 0.95,
)
list(zip(solutions[1], solutions[0]))[::-7]

['z4', "x4'", 'z4', 'z4', 'y4', "z4'", "y4'", "z0'", 'x4', "y4'", 'y0']


[(0.35310855259289087,
  (("x0'", "x0'", "z4'", "y0'", 'x0', 'z4', "y4'", "z4'", 'z0'),
                  4  1  2  3  4 
                  19 18 17 16 23
                  14 13 12 11 22
                  9  8  7  6  21
                  24 19 14 9  4 
   20 15 10 5  0  20 1  2  3  0  24 15 10 5  4  0  1  2  3  0 
   3  8  13 18 9  5  6  7  8  15 19 6  7  8  23 3  8  13 18 15
   2  7  12 17 14 10 11 12 13 10 14 11 12 13 22 2  7  12 17 10
   1  6  11 16 19 15 16 17 18 5  9  16 17 18 21 1  6  11 16 5 
   0  19 14 9  4  24 21 22 23 20 20 3  2  1  24 20 19 14 9  4 
                  20 23 22 21 24
                  5  6  7  8  23
                  10 11 12 13 22
                  15 16 17 18 21
                  0  21 22 23 24)),
 (0.5335822239583333,
  (('x0', "z4'", 'x0', 'x0', 'z0'),
                  20 1  2  3  4 
                  3  16 11 6  9 
                  2  17 12 7  14
                  1  18 13 8  19
                  4  5  10 15 0 
   4  23 22 21 0  0  1  2  3  20 24 15 10

In [5]:
c.possible_moves

['x0', "x0'", 'y0', "y0'", 'z0', "z0'", 'x2', "x2'", 'y2', "y2'", 'z2', "z2'"]